In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-06 13:54:40,393 INFO: Initializing external client
2025-03-06 13:54:40,395 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:54:41,463 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Fetching data from 2025-02-05 18:54:40.390918+00:00 to 2025-03-06 17:54:40.390918+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 13:54:50,703 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 13:54:50,711 INFO: Initializing external client
2025-03-06 13:54:50,712 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:54:51,867 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681


In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [7]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,114,25.0,2025-03-06 19:00:00+00:00
1,165,0.0,2025-03-06 19:00:00+00:00
2,93,-0.0,2025-03-06 19:00:00+00:00
3,263,31.0,2025-03-06 19:00:00+00:00
4,241,1.0,2025-03-06 19:00:00+00:00
...,...,...,...
247,94,0.0,2025-03-06 19:00:00+00:00
248,174,0.0,2025-03-06 19:00:00+00:00
249,185,-0.0,2025-03-06 19:00:00+00:00
250,37,1.0,2025-03-06 19:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-06 13:54:54,660 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 13:54:54,667 INFO: Initializing external client
2025-03-06 13:54:54,669 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:54:55,750 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 252/252 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215681/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)

In [12]:
import hsfs

# Create a connection to the Hopsworks feature store
# The parameters depend on your Hopsworks setup and whether you are using it inside Hopsworks or externally
conn = hsfs.connection(
    host='my-instance.cloud.hopsworks.ai',
    project='ejeshkr',
    api_key_value='r25PmHdLuyYBdqBS.uKTU5BdMQB7y45O6fhUyJMqFiQkutEAUTQI1FxGLvebMknsXTtqGvf7I4ooA8cMJ',  # Your API key for authentication
    port=443,  # Default HTTPS port for Hopsworks instances
    hostname_verification=True
)

# Get the feature store handle
feature_store = conn.get_feature_store()


2025-03-06 13:57:11,980 INFO: Initializing external client
2025-03-06 13:57:11,981 INFO: Base URL: https://https://pypi.org/project/hopsworks/4.1.5/ :443


ValueError: Invalid port ''.